In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
from transformers import AutoTokenizer


from lxt.explicit.models.llama import LlamaForCausalLM, attnlrp
from lxt.utils import pdf_heatmap, clean_tokens
import transformers

# assert torch.__version__.startswith("2.1"), "PyTorch version must be 2.1. Code might not work with newer versions."
# assert transformers.__version__.startswith("4.46"), "Transformers version must be 4.46.2 Code might not work with newer versions."

path = 'meta-llama/Llama-3.2-1B-Instruct'

model = LlamaForCausalLM.from_pretrained(path, torch_dtype=torch.bfloat16, device_map="cuda")
tokenizer = AutoTokenizer.from_pretrained(path)

# apply AttnLRP rules
attnlrp.register(model)

prompt = """\
Context: Mount Everest attracts many climbers, including highly experienced mountaineers. There are two main climbing routes, one approaching the summit from the southeast in Nepal (known as the standard route) and the other from the north in Tibet. While not posing substantial technical climbing challenges on the standard route, Everest presents dangers such as altitude sickness, weather, and wind, as well as hazards from avalanches and the Khumbu Icefall. As of November 2022, 310 people have died on Everest. Over 200 bodies remain on the mountain and have not been removed due to the dangerous conditions. The first recorded efforts to reach Everest's summit were made by British mountaineers. As Nepal did not allow foreigners to enter the country at the time, the British made several attempts on the north ridge route from the Tibetan side. After the first reconnaissance expedition by the British in 1921 reached 7,000 m (22,970 ft) on the North Col, the 1922 expedition pushed the north ridge route up to 8,320 m (27,300 ft), marking the first time a human had climbed above 8,000 m (26,247 ft). The 1924 expedition resulted in one of the greatest mysteries on Everest to this day: George Mallory and Andrew Irvine made a final summit attempt on 8 June but never returned, sparking debate as to whether they were the first to reach the top. Tenzing Norgay and Edmund Hillary made the first documented ascent of Everest in 1953, using the southeast ridge route. Norgay had reached 8,595 m (28,199 ft) the previous year as a member of the 1952 Swiss expedition. The Chinese mountaineering team of Wang Fuzhou, Gonpo, and Qu Yinhua made the first reported ascent of the peak from the north ridge on 25 May 1960. \
Question: How high did they climb in 1922? According to the text, the 1922 expedition reached 8,"""

# get input embeddings so that we can compute gradients w.r.t. input embeddings
input_ids = tokenizer(prompt, return_tensors="pt", add_special_tokens=True).input_ids.to(model.device)
input_embeds = model.get_input_embeddings()(input_ids)

# inference and get the maximum logit at the last position
output_logits = model(inputs_embeds=input_embeds.requires_grad_(), use_cache=False).logits
max_logits, max_indices = torch.max(output_logits[0, -1, :], dim=-1)

# get the top k tokens and their logits
topk_logits, topk_indices = torch.topk(output_logits[0, -1, :], k=5, dim=-1)

# convert token indices to strings
topk_tokens = tokenizer.convert_ids_to_tokens(topk_indices)

In [ ]:
torch.cuda.max_memory_allocated()

In [ ]:
# print the top k tokens and their logits
for token, logit in zip(topk_tokens, topk_logits):
    print(f"Token: {token}, Logit: {logit.item()}")

# initialize relevance scores with max_logits itself and backpropagate
max_logits.backward(max_logits)
relevance = input_embeds.grad.float().sum(-1).cpu()[0] # cast to float32 before summation for higher precision

# normalize relevance between [-1, 1] for plotting
relevance = relevance / relevance.abs().max()

# remove '_' characters from token strings and plot the heatmap
tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
tokens = clean_tokens(tokens)

print(relevance.topk(5))

pdf_heatmap(tokens, relevance, path='attnlrp_heatmap.pdf', backend='xelatex')

In [ ]:
relevance.shape

In [ ]:
torch.cuda.max_memory_allocated()

# Try our LRP now (restart kernel and run from here)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from lxt.utils import pdf_heatmap, clean_tokens
import os
import sys
sys.path.append('../../src')

In [3]:
path = 'meta-llama/Llama-3.2-1B-Instruct'

model = AutoModelForCausalLM.from_pretrained(path, device_map="cuda", attn_implementation="eager")
tokenizer = AutoTokenizer.from_pretrained(path)

prompt = """\
Context: Mount Everest attracts many climbers, including highly experienced mountaineers. There are two main climbing routes, one approaching the summit from the southeast in Nepal (known as the standard route) and the other from the north in Tibet. While not posing substantial technical climbing challenges on the standard route, Everest presents dangers such as altitude sickness, weather, and wind, as well as hazards from avalanches and the Khumbu Icefall. As of November 2022, 310 people have died on Everest. Over 200 bodies remain on the mountain and have not been removed due to the dangerous conditions. The first recorded efforts to reach Everest's summit were made by British mountaineers. As Nepal did not allow foreigners to enter the country at the time, the British made several attempts on the north ridge route from the Tibetan side. After the first reconnaissance expedition by the British in 1921 reached 7,000 m (22,970 ft) on the North Col, the 1922 expedition pushed the north ridge route up to 8,320 m (27,300 ft), marking the first time a human had climbed above 8,000 m (26,247 ft). The 1924 expedition resulted in one of the greatest mysteries on Everest to this day: George Mallory and Andrew Irvine made a final summit attempt on 8 June but never returned, sparking debate as to whether they were the first to reach the top. Tenzing Norgay and Edmund Hillary made the first documented ascent of Everest in 1953, using the southeast ridge route. Norgay had reached 8,595 m (28,199 ft) the previous year as a member of the 1952 Swiss expedition. The Chinese mountaineering team of Wang Fuzhou, Gonpo, and Qu Yinhua made the first reported ascent of the peak from the north ridge on 25 May 1960. \
Question: How high did they climb in 1922? According to the text, the 1922 expedition reached 8,"""

# get input embeddings so that we can compute gradients w.r.t. input embeddings
input_ids = tokenizer(prompt, return_tensors="pt", add_special_tokens=True).input_ids.to(model.device)
input_embeds = model.get_input_embeddings()(input_ids)


In [4]:
# inference and get the maximum logit at the last position
output_logits = model(input_ids).logits
max_logits, max_indices = torch.max(output_logits[0, -1, :], dim=-1)

# get the top k tokens and their logits
topk_logits, topk_indices = torch.topk(output_logits[0, -1, :], k=5, dim=-1)

# convert token indices to strings
topk_tokens = tokenizer.convert_ids_to_tokens(topk_indices)

In [5]:
topk_tokens

['320', '300', '000', '280', 'Ġ']

In [6]:
from lrp_engine import LRPEngine

In [7]:
lrp = LRPEngine(use_attn_lrp=True)

In [8]:
lrp_output = lrp.run(max_logits.unsqueeze(0))

In [9]:
relevance = lrp_output[1][0][0].float().cpu()

In [10]:
topk_logits, topk_indices = relevance.topk(15)
topk_tokens = tokenizer.convert_ids_to_tokens(input_ids[0][topk_indices])

In [11]:
topk_tokens

[',',
 '320',
 'Ġ(',
 'Ġft',
 '595',
 '2',
 'Ġroute',
 '000',
 '27',
 '000',
 'Ġclimb',
 '970',
 ',',
 '.',
 'Ġmade']

In [12]:
# normalize relevance between [-1, 1] for plotting
relevance = relevance / relevance.abs().max()

In [13]:
# remove '_' characters from token strings and plot the heatmap
tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
tokens = clean_tokens(tokens)


In [14]:
pdf_heatmap(tokens, relevance, path='ourlrp_eager_heatmap_f32.pdf', backend='xelatex')

PDF file generated successfully.
